In [1]:
#pour réduire le temps de prédiction, prédire sur chacun des stores en parallèle sur
#plusieurs notebook et ensuire concat les resultats


In [1]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
from tqdm import tqdm, tnrange
from multiprocessing import Pool, cpu_count

In [43]:
calendar_df = pd.read_csv('../../data/calendar.csv')
sales_train =  pd.read_csv('../../data/sales_train_validation.csv')



In [44]:
# permet de savoir si on se trouve dans la periode des saisons de NFL
def is_nfl_season(ds):
    date = pd.to_datetime(ds)
    return (date.month > 8 or date.month < 2)

In [45]:
def run_prophet(timeserie):
    #activer le yearly_seasonality
    model = Prophet(uncertainty_samples=False,yearly_seasonality=20, weekly_seasonality=False)
    #tuning avec les holidays des US de prophet, possibilité de mettre ses propres holidays
    model.add_country_holidays(country_name='US')
    #ajouter une seasonality sur le mois en prennant 1 mois = 28 jours
    model.add_seasonality(name='monthly', period=28, fourier_order=5)
    model.add_seasonality(name='weekly_on_season', period=7, fourier_order=3, condition_name='on_season')
    model.add_seasonality(name='weekly_off_season', period=7, fourier_order=3, condition_name='off_season')
    
    model.fit(timeserie)
    future = model.make_future_dataframe(periods=56, include_history=False)
    future['on_season'] = future['ds'].apply(is_nfl_season)
    future['off_season'] = ~future['ds'].apply(is_nfl_season)
    forecast = model.predict(future)
    return forecast

In [46]:
sales_train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [54]:
sales_train_store = sales_train.loc[(sales_train.store_id == "WI_2") ]

In [55]:
sales_train_store

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
24392,HOBBIES_1_001_WI_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
24393,HOBBIES_1_002_WI_2_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
24394,HOBBIES_1_003_WI_2_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
24395,HOBBIES_1_004_WI_2_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,0,2,0,2,0,1,0,2,1,0
24396,HOBBIES_1_005_WI_2_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,4,2,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27436,FOODS_3_823_WI_2_validation,FOODS_3_823,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
27437,FOODS_3_824_WI_2_validation,FOODS_3_824,FOODS_3,FOODS,WI_2,WI,1,0,0,0,...,1,2,0,0,0,0,2,0,1,0
27438,FOODS_3_825_WI_2_validation,FOODS_3_825,FOODS_3,FOODS,WI_2,WI,0,3,0,2,...,3,0,1,2,6,0,2,2,0,2
27439,FOODS_3_826_WI_2_validation,FOODS_3_826,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,1,2,4,3,0,0,4,1,1,1


In [49]:
file_sub = sales_train_store.copy()
file_sub = file_sub[['id']]
for i in range(1,57):
    file_sub[f"F{i}"] = 0.0
file_sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_005_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3045,FOODS_3_824_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3046,FOODS_3_825_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3047,FOODS_3_826_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
#la boucle fait une prediction en fonction de l'index, qui represente 1 items dans l'ordre
# dans lequel ils sont dans la df sales_train

start_from_ob = 1200

#sales_train.shape[0]
for i in tnrange(2):
    temp_series = sales_train_store.iloc[i,start_from_ob:]
    temp_series.index = calendar_df['date'][start_from_ob:start_from_ob+len(temp_series)]
    temp_series =  pd.DataFrame(temp_series)
    temp_series = temp_series.reset_index()
    temp_series.columns = ['ds', 'y']
    
    #pour savoir si c'est la saison de NFL
    temp_series['on_season'] = temp_series['ds'].apply(is_nfl_season)
    temp_series['off_season'] = ~temp_series['ds'].apply(is_nfl_season)

    
    with Pool(cpu_count()) as p:
        forecast1 = p.map(run_prophet, [temp_series])
    #print(forecast1[0]['yhat'].values)
    file_sub.iloc[i,1:] = forecast1[0]['yhat'].values

    file_sub.iloc[:,1:]=file_sub.iloc[:,1:].where(file_sub.iloc[:,1:] > 0).fillna(0)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [51]:
file_sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56
0,HOBBIES_1_001_CA_1_validation,0.732421,0.921347,0.766197,0.694087,0.858233,1.145177,0.677165,0.478139,0.535143,...,0.584585,0.931858,0.482489,0.321813,0.485271,0.369309,0.377275,0.617097,0.963158,0.572129
1,HOBBIES_1_002_CA_1_validation,0.203215,0.191428,0.050981,0.103415,0.195108,0.328457,0.451474,0.288894,0.249132,...,0.547678,0.351795,0.275832,0.179040,0.320551,0.353149,0.421032,0.372960,0.374472,0.502999
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3045,FOODS_3_824_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3046,FOODS_3_825_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3047,FOODS_3_826_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
